In [ ]:
# Author : Raude Killian
# Last modified 16.07.205
# Last Check by Qiaoyue 15.07.2025

In [ ]:
import mne
import matplotlib.pyplot as plt
import pandas as pd
import os
import warnings
%matplotlib inline

PARTICIPANTS_FILE = r"L:\Common\Users\Qiaoyue\MEG_project\Data\participants.csv"

warnings.filterwarnings("ignore", category=RuntimeWarning)
warnings.filterwarnings("ignore", category=UserWarning)

participants = pd.read_csv(PARTICIPANTS_FILE)  
participants = participants['subID'].astype(str).tolist()
sessions = ["encoding", "recall"]

print(f"✅ {len(participants)} participants list : {participants}")

In [ ]:
# For tests purposes
#participants = ["F103"]
#sessions = ["encoding"]

In [ ]:
#Run the code once and examine each components for each subjects for each session to decide which one to remove.

for subID in participants:
    for session in sessions:
        plots_folder = rf"L:\Common\Users\Qiaoyue\MEG_project\Results\plots\{subID}"
        os.makedirs(plots_folder, exist_ok=True)
        
        #intermediates_folder = rf"L:\Common\Users\Qiaoyue\MEG_project\Results\meg_intermediates\{subID}"
        intermediates_folder = rf"C:\Users\killg\Desktop\meg_intermediates\{subID}"
        os.makedirs(intermediates_folder, exist_ok=True)

        print(f"Preprocessing: {subID}_{session}_session")
        
        #Declare filename
        sss_meg_filename = rf"{intermediates_folder}/{subID}_{session}_sss.fif"
        sss_meg_file = mne.io.read_raw_fif(sss_meg_filename, preload=True, verbose=False)

        sss_meg_file_filtered = sss_meg_file.copy().pick('meg')

        # Downsample, ICA does not need high resolution.
        sss_meg_file_filtered.resample(200, verbose=False) 

        # The 1 Hz highpass is important for removing slow drifts which otherwise would make the ICA decomposition less efficient.
        sss_meg_file_filtered.filter(1, 40, verbose=False)

        ica = mne.preprocessing.ICA(n_components=0.99, method='fastica', random_state=1906, verbose=False) 
        ica.fit(sss_meg_file_filtered, verbose=False)
        
        # Save figures
        figs = ica.plot_components(show=False)
        for i, fig in enumerate(figs):
            fig_path = os.path.join(plots_folder, f"{subID}_{session}_ICA_component_{i+1}.png")
            fig.savefig(fig_path)
            plt.close(fig)
        
        print(f"✅ {subID}_{session}_session sucessfully completed  \n")